In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from glue import BlindNavigator
from utility import read_video
from PIL import Image
import numpy as np
import cv2
import time

class Visor(object):
    def __init__(self, form = 'OFFLINE'):
        self.form = form
        self.handler = None
        self.handler2 = None
        
    def initializer(self):
        if self.form == 'OFFLINE':
            self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
            writer = None
            (h, w) = (None, None)
            self.handler = writer 
            
        else:
            fig = plt.figure(figsize=(16, 10))
            ax = fig.add_axes([0,0,1,1])

            plt.ion()
            fig.show()
            fig.canvas.draw()
            self.handler = fig
            self.handler2 = ax

            
    def inliner(self, im, save_path='./test.avi'):
        if self.form == 'OFFLINE' and self.handler is None:
            (h, w) = im.shape[:2]
            self.handler = cv2.VideoWriter(save_path, self.fourcc, 30, (w, h), True)
    
    def drawer(self, im):
        if self.form == 'OFFLINE':
            self.handler.write(im)
        else:
            self.handler2.imshow(im)
            self.handler.canvas.draw()
            self.handler2.clear()
debuger = 'OFFLINE'
def test_det_track():
    video_path = 'data/IMG_1051.MOV'
    save_path = 'IMG_1051_det_track.avi'

    visor = Visor(debuger)
    visor.initializer()

    navigator = BlindNavigator()
    timestamps = [time.time()]
    for data in read_video(video_path):
        timestamps.append(time.time())

        traffic_lights, detected_obstacles = navigator.detect_traffic_light(data)
        timestamps.append(time.time())
        print('Processing time: %.3f'%(timestamps[-1] - timestamps[-2]))
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        visor.inliner(data, save_path)
        for (id_, bbox) in traffic_lights:
            
            bbox = bbox.astype(int)
            assert  id_ < len(navigator.traffic_light_pool.trackers), 'id_ not valid'
#             bbox = [bbox[0] / data.shape[1], bbox[1] / data.shape[0], bbox[2] / data.shape[1], bbox[3] / data.shape[0]]
#             print(bbox)
#             p = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], fill=False, alpha=1)
#             ax.add_patch(p)
            cv2.rectangle(data, tuple(bbox[:2]),tuple(bbox[2:4]), [255, 0, 0], 3)
        visor.drawer(data)

def test_light_classifier():
    video_path = 'data/IMG_1051.MOV'
    save_path = 'IMG_1051_det_track_cls.avi'

    visor = Visor(debuger)
    visor.initializer()
    navigator = BlindNavigator()
    for data in read_video(video_path):
        traffic_lights, detected_obstacles = navigator.detect_traffic_light(data)
        light_states = navigator.color_classify_by_boxes(data, [light[1] for light in traffic_lights])
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        visor.inliner(data, save_path)
        for (id_, bbox), state in zip(traffic_lights, light_states):
            
            bbox = bbox.astype(int)
            assert  id_ < len(navigator.traffic_light_pool.trackers), 'id_ not valid'
#             bbox = [bbox[0] / data.shape[1], bbox[1] / data.shape[0], bbox[2] / data.shape[1], bbox[3] / data.shape[0]]
#             print(bbox)
#             p = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], fill=False, alpha=1)
#             ax.add_patch(p)
            cv2.rectangle(data, tuple(bbox[:2]),tuple(bbox[2:4]), [255, 0, 0], 3)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(data, '%s'%(state), (bbox[0]-1,bbox[1]-1), font, 1, color=(255,255,0), thickness=2)
        visor.drawer(data)


        
        
        
if __name__ == '__main__':
    test_det_track()

/home/changcheng/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


keras_yolo3/model_data/yolo.h5 model, anchors, and classes loaded.
get frame 0
[0.2054599  0.0137211  0.01778067 ... 0.19325048 0.09076633 0.08210257]
[0.         0.00511656 0.00028194 ... 0.14154416 0.13942064 0.13884631]
[0.4        0.         0.         ... 0.19057007 0.15867716 0.1281939 ]
[0.         0.         0.         ... 0.12139461 0.12727863 0.12811518]
[0.         0.         0.         ... 0.12570392 0.16845728 0.12281688]
Processing time: 3.419
get frame 1
[0.17526558 0.0016417  0.00219152 ... 0.17639543 0.08928764 0.08293206]
[0.02011634 0.         0.02837306 ... 0.1147473  0.10500809 0.10616661]
[0.4        0.09498732 0.00423655 ... 0.24538743 0.06657743 0.05962665]
[0.0107201  0.00045561 0.00855878 ... 0.14880538 0.14800116 0.14576644]
[0.08483052 0.05292986 0.         ... 0.16334932 0.1662525  0.16147214]
[0.00175221 0.         0.01780435 ... 0.19398238 0.23697749 0.17318474]
[0.4        0.06402964 0.         ... 0.20460775 0.17759801 0.13419189]
[0.3189848  0.02916615

[0.         0.         0.         ... 0.14803527 0.160458   0.16058032]
[0.         0.         0.         ... 0.09470044 0.1397967  0.17424445]
[0.00059424 0.         0.         ... 0.1383461  0.15642351 0.14224587]
[0.1680376  0.         0.0130572  ... 0.11668392 0.11896212 0.10773389]
[0.38745746 0.02342767 0.08890175 ... 0.11508793 0.16312867 0.11813826]
[0.01995618 0.00405667 0.02565664 ... 0.11328233 0.0835266  0.11476981]
Processing time: 0.168
get frame 9
[0.26225662 0.4        0.01384581 ... 0.12227266 0.11963499 0.11066208]
[0.01900364 0.06931445 0.         ... 0.16327609 0.16450027 0.14984357]
[0.38836    0.38116026 0.0811105  ... 0.16036533 0.14754623 0.12952307]
[0.18732764 0.08900606 0.00338102 ... 0.14649148 0.1631691  0.13756964]
[0.33063933 0.10357612 0.02419856 ... 0.12993795 0.1847422  0.12705459]
[0.00574071 0.06416071 0.         ... 0.17677152 0.19640698 0.1793036 ]
[0.03203097 0.         0.04529862 ... 0.18481119 0.14047475 0.10907451]
[0.         0.         0.0081

[0.0495621  0.0443297  0.0233638  ... 0.24864416 0.1490659  0.10902076]
[0.2048133  0.06702255 0.24680442 ... 0.16809537 0.14808805 0.10861723]
[0.00503517 0.         0.         ... 0.14029609 0.09439477 0.10821906]
[0.         0.01055061 0.         ... 0.13280284 0.14445573 0.14359123]
[0.         0.         0.0007352  ... 0.08380086 0.11975557 0.15270546]
[0.         0.         0.         ... 0.13597615 0.15662298 0.14029945]
[0.         0.         0.         ... 0.09567003 0.10991546 0.09887772]
[0.         0.         0.         ... 0.12809722 0.16588178 0.12632236]
[0.         0.         0.         ... 0.14023621 0.09754246 0.14773503]
Processing time: 0.200
get frame 17
[0.37366605 0.18650696 0.27599695 ... 0.12412189 0.10937275 0.10205312]
[0.03125095 0.02164584 0.03189034 ... 0.1573645  0.15907942 0.14295502]
[0.38679203 0.28179085 0.18418369 ... 0.1714814  0.15105645 0.12944202]
[0.00846991 0.03151792 0.00305096 ... 0.16707234 0.19504434 0.17496428]
[0.0895246  0.17224196 0.   

[0.19706365 0.09438064 0.05377734 ... 0.156869   0.16420476 0.15334949]
[0.00469541 0.00316476 0.         ... 0.19877592 0.25532404 0.20955753]
[0.12323888 0.02922717 0.         ... 0.27117586 0.1596481  0.1083079 ]
[0.31236875 0.07480638 0.10976315 ... 0.28319603 0.15549402 0.10373086]
[0.01430107 0.00456831 0.         ... 0.2026039  0.12256906 0.15724444]
[0.         0.01518487 0.00962492 ... 0.11365668 0.11746575 0.11748186]
[0.         0.         0.         ... 0.08555263 0.1145803  0.13621646]
[0.         0.00515347 0.         ... 0.13686363 0.1561526  0.1402272 ]
[0.         0.         0.         ... 0.11052268 0.18456909 0.1071085 ]
[0.         0.         0.         ... 0.09847718 0.12350329 0.09851445]
[0.         0.         0.         ... 0.10971344 0.08766449 0.10817209]
Processing time: 0.175
get frame 25
[0.19697356 0.18968296 0.16632666 ... 0.14343026 0.14547698 0.1332859 ]
[0.00495796 0.01287141 0.01311607 ... 0.14255504 0.15467745 0.14536686]
[0.20139664 0.21146005 0.066

...

[0.15956563 0.         0.21474084 ... 0.15881158 0.20795503 0.16026135]
[0.         0.         0.         ... 0.11072738 0.11272051 0.11304533]
[0.00069505 0.         0.         ... 0.10641254 0.12173469 0.12945788]
[0.         0.00128258 0.         ... 0.11800126 0.12401021 0.1182183 ]
[0.         0.         0.         ... 0.1421332  0.15377218 0.14074205]
[0.         0.         0.         ... 0.09505406 0.09623495 0.09296509]
[0.         0.         0.         ... 0.1303129  0.12679558 0.11536069]
[0.         0.         0.         ... 0.15902996 0.15914334 0.08660954]
[0.03591277 0.01480676 0.         ... 0.15069225 0.14235677 0.10178638]
[0.         0.         0.         ... 0.12124729 0.1413627  0.10623177]
Processing time: 0.157
get frame 258
[0.3809497  0.33371857 0.09436517 ... 0.14075801 0.12908842 0.11812647]
[0.08703865 0.06683207 0.01233394 ... 0.17849463 0.14628686 0.14889881]
[0.4        0.18742424 0.02758032 ... 0.15904385 0.12019555 0.11342385]
[0.         0.00271936 0.  

[0.4        0.         0.03619519 ... 0.13858129 0.11099371 0.11082801]
[0.01268425 0.         0.         ... 0.11839574 0.12613238 0.12531698]
[0.         0.         0.         ... 0.1020941  0.13684678 0.16207442]
[0.         0.         0.         ... 0.12879193 0.14192148 0.13210638]
[0.         0.         0.         ... 0.11300415 0.14283262 0.11422825]
[0.         0.         0.         ... 0.09352022 0.10207677 0.09628157]
[0.04113594 0.         0.01939167 ... 0.11307342 0.1453934  0.10225751]
[0.01761343 0.0019084  0.00521002 ... 0.15654805 0.14148857 0.09974274]
[0.01068131 0.05170408 0.03915896 ... 0.16456328 0.14728603 0.1283839 ]
[0.         0.         0.         ... 0.1407966  0.14638261 0.09025759]
Processing time: 0.146
get frame 265
[0.4        0.17833069 0.05646193 ... 0.1637021  0.14647797 0.13244128]
[0.04256547 0.07732908 0.05246    ... 0.18003297 0.16618064 0.15194666]
[0.4        0.18286236 0.0522242  ... 0.17993212 0.14185818 0.12728027]
[0.02577155 0.00467094 0.00

[0.         0.         0.         ... 0.10517693 0.12105709 0.13554148]
[6.1792140e-05 0.0000000e+00 0.0000000e+00 ... 1.6392066e-01 2.0023303e-01
 1.6995870e-01]
[0.3713947  0.15367092 0.05102476 ... 0.12148876 0.15592025 0.11785992]
[0.13890034 0.         0.04388436 ... 0.12093421 0.12708434 0.12093943]
[0.4        0.07478531 0.07432604 ... 0.10891671 0.09592769 0.10391089]
[0.01233917 0.02200645 0.0380762  ... 0.16902933 0.18867567 0.12510978]
[0.01457878 0.06639796 0.03517934 ... 0.16058654 0.17337166 0.1649433 ]
[0.00854479 0.         0.         ... 0.135584   0.1428784  0.10025033]
Processing time: 0.172
get frame 272
[0.09578251 0.08757329 0.04348661 ... 0.17161907 0.15531138 0.16062023]
[0.00105013 0.00323994 0.         ... 0.19079639 0.1538247  0.15258212]
[0.3861253  0.06971174 0.16877301 ... 0.13136649 0.12322453 0.120757  ]
[0.02699528 0.02514059 0.00650661 ... 0.21567568 0.23530841 0.22076479]
[0.2553599  0.08459371 0.         ... 0.17198277 0.2213042  0.1665191 ]
[0.     

[0.         0.02334648 0.         ... 0.09761196 0.09988575 0.07602207]
Processing time: 0.139
get frame 278
[0.3922723  0.3530557  0.07010197 ... 0.12726998 0.12042256 0.1180537 ]
[0.01123973 0.09260231 0.04755643 ... 0.1750694  0.16879216 0.16630323]
[0.354802   0.08498931 0.1214858  ... 0.15776968 0.12060312 0.11601437]
[0.08127949 0.00213601 0.         ... 0.19068703 0.2153966  0.20093846]
[0.25028205 0.00217935 0.         ... 0.15805314 0.22663984 0.15028511]
[0.00592574 0.00140482 0.         ... 0.22859567 0.287379   0.24024434]
[0.08518479 0.         0.39523363 ... 0.24141276 0.17888997 0.17965384]
[0.05779678 0.         0.14178866 ... 0.2893094  0.17733897 0.17657411]
[0.26480106 0.04585074 0.28843462 ... 0.19817114 0.1410859  0.14034975]
[0.         0.         0.         ... 0.16469394 0.17703746 0.17669514]
[0.         0.         0.         ... 0.11071119 0.14806493 0.16332178]
[0.00361454 0.         0.         ... 0.14948353 0.16615553 0.15332049]
[0.38609105 0.05862563 0.03

get frame 285
[0.317288   0.2191046  0.20546815 ... 0.14356339 0.12610005 0.11952394]
[0.08261282 0.08064739 0.05545393 ... 0.18009976 0.18232605 0.16941912]
[0.37635383 0.2207918  0.126569   ... 0.15517196 0.12045478 0.11879338]
[0.08055399 0.02957674 0.         ... 0.22546604 0.25206986 0.24111015]
[0.29272243 0.0156268  0.0512273  ... 0.19818519 0.22727527 0.17922886]
[0.         0.00354741 0.00346455 ... 0.21002257 0.25414574 0.22044863]
[0.4        0.4        0.13861878 ... 0.20754576 0.21860883 0.20573975]
[0.4        0.35661277 0.30240738 ... 0.27364278 0.22375336 0.2179529 ]
[0.36715427 0.22023878 0.19683993 ... 0.18779033 0.18137084 0.17145781]
[0.         0.         0.         ... 0.11361744 0.11792025 0.11774592]
[0.         0.03265253 0.         ... 0.09544119 0.11358825 0.13341795]
[0.00065964 0.         0.         ... 0.14363904 0.15789215 0.14759512]
[0.4        0.14475511 0.02080709 ... 0.12462188 0.16481149 0.12073015]
[0.4        0.11997807 0.03902441 ... 0.09478988 0